<h4>TextCNN</h4>

<h5>Imports</h5>

In [1]:
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse, unquote_plus
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Input, Dropout
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
import jsonlines
import os
import json
import pickle
from sklearn.metrics import f1_score, accuracy_score, precision_score
from typing import Callable
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from tensorflow.python.profiler import profiler_v2 as profiler

2023-05-16 17:36:35.082959: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-16 17:36:35.133897: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 17:36:35.701803: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#loda all payloads from txt files in dataset folder (where each payload is one line)
def load_payloads(dataset_path: str) -> list:
    payloads = []
    for filename in os.listdir(dataset_path):
        with open(dataset_path + filename, 'r') as f:
            payloads.extend(f.readlines())
    return payloads

In [3]:
file_names=['malicious','normal']
for name in file_names:
    data=[]
    with jsonlines.open('dataset/'+name+'.txt') as reader:
        for line in reader:
            if line['request']['method'] == 'POST':
                data.append({'request':{'method':'POST', 'uri':line['request']['uri'], 'body':line['request']['body'],'headers':line['request']['headers']},'metadata':line['metdata']})
            else:
                data.append({'request':{'method':'GET', 'uri':line['request']['uri'], 'headers':line['request']['headers']},'metadata':line['metdata']})
    with jsonlines.open('dataset/'+name+'_clean.txt', mode='w') as writer:
        writer.write_all(data)

<h4>Load and create datasets</h4>

In [4]:
from collections import Counter


file_names = ['dataset/malicious_clean.txt' ,'dataset/normal_clean.txt']
data={}
for file in file_names:
    data[file]=[]
    with jsonlines.open(file) as reader:
        for line in reader:
            if line['request']['method'] == 'POST':
                data[file].append('POST'+' '+line['request']['uri']+' '+line['request']['body']+' '+json.dumps(line['request']['headers']))
            else:
                data[file].append('GET'+' '+line['request']['uri']+' '+json.dumps(line['request']['headers']))

payloads = load_payloads('dataset/payloads/')      
                
normal = data[file_names[1]]
malicious = data[file_names[0]]
normal_part1 = normal[180000:]
normal_part2 = normal[:180000]

train_examples = normal_part2
test_examples = normal_part1+malicious
train_labels = [0] * len(train_examples)
test_labels = [0]* len(normal_part1)
test_labels.extend([1] * len(malicious))

payload_iterator={}
payload_iterator[0] = 0
def get_next_payload():
    payload_iterator[0] = (payload_iterator[0] + 1) % len(payloads)
    return payloads[payload_iterator[0]]

#create synthetic malicious data by adding payloads to normal data (PAYLOAD SHOULD BE injected at a space character )
tmp = normal_part1.copy()
np.random.shuffle(payloads)
synthetic_malicious = []
for i in range(len(tmp)):
    payload = get_next_payload()
    tmp[i] = tmp[i].split(' ') 
    tmp[i].insert(np.random.randint(0,len(tmp[i])), payload)
    tmp[i] = ' '.join(tmp[i])
    synthetic_malicious.append(tmp[i])


# add shuffled malicious data to train set
#malicious_shuffled = malicious.copy()  
#np.random.shuffle(malicious_shuffled)
#train_examples.extend(malicious_shuffled[:50000])
#train_labels.extend([1] * len(malicious_shuffled[:50000]))

train_examples.extend(synthetic_malicious)
train_labels.extend([1] * len(synthetic_malicious))

dataset_train = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),train_examples)), train_labels))
dataset_test = tf.data.Dataset.from_tensor_slices((list(map(lambda x: unquote_plus(x),test_examples)), test_labels))

#shuflee datasets with given radnom seed
dataset_train = dataset_train.shuffle(400000, seed=42, reshuffle_each_iteration=False)
dataset_test = dataset_test.shuffle(400000, seed=42, reshuffle_each_iteration=False)

def preprocess_text(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    text = tf.strings.regex_replace(text, '(.)',  r'\1 ')
    return text, label

def preprocess_text_substitution(text, label):
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, "[a-zA-Z]+", "a")
    text = tf.strings.regex_replace(text, "[0-9]+", "n")
    text = tf.strings.regex_replace(text, "(a|n){2,}", "x")
    text = tf.strings.regex_replace(text, '[^\x00-\x7F]+', '')
    punctuation = "=?/(){}[]<>"
    for p in punctuation:
        text = tf.strings.regex_replace(text, "\\" + p, " "+p+" ")
    return text, label

 
# Map the preprocess function to the dataset
train = dataset_train.map(preprocess_text)
test = dataset_test.map(preprocess_text)

train_sub = dataset_train.map(preprocess_text_substitution)
test_sub = dataset_test.map(preprocess_text_substitution)

for text, label in train_sub.take(1):
    print(text.numpy(), label.numpy())

for text, label in train.take(1):
    print(text.numpy(), label.numpy())




2023-05-16 17:36:51.204051: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 17:36:51.247305: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 17:36:51.247360: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 17:36:51.249166: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-16 17:36:51.249214: I tensorflow/compile

b'a  / a / a-a  { "a-a": "a-a", "a": "a / a,a / a a,a / a;a = n.n,a / a,a / a,* / *;a = n.n", "a-a": "a", "a-a": "a / n.n  ( a; a a a a n_n_n )  a / n.n  ( a, a a )  a / n.n.n.n a / n.n", "a": "a = x; a = x" } ' 0
b'a   / a / a - a   { " a - a " :   " a - a " ,   " a " :   " a / a , a / a   a , a / a ; a = n . n , a / a , a / a , * / * ; a = n . n " ,   " a - a " :   " a " ,   " a - a " :   " a / n . n   ( a ;   a   a   a   a   n _ n _ n )   a / n . n   ( a ,   a   a )   a / n . n . n . n   a / n . n " ,   " a " :   " a = x ;   a = x " } ' 0


2023-05-16 17:36:53.478697: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [211600]
	 [[{{node Placeholder/_1}}]]


<h4>Vocabularies</h4>

In [34]:

vectorization = TextVectorization(
    max_tokens=1000,
    output_mode="int", 
    output_sequence_length=256,
    pad_to_max_tokens=True,
    standardize="lower"
)

vectorization_sub = TextVectorization(
    max_tokens=5000, 
    output_mode="int", 
    output_sequence_length=128,
    pad_to_max_tokens=True,
    standardize="lower"
)

vectorization.adapt(train.map(lambda x, y: x))
vectorization_sub.adapt(train_sub.map(lambda x, y: x))

2023-04-23 00:43:23.850876: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]
2023-04-23 00:53:16.487408: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int32 and shape [180000]
	 [[{{node Placeholder/_1}}]]


<h4>Save and load vectorization from disk</h4>

In [5]:
#pickle.dump({'config': vectorization.get_config(),'weights': vectorization.get_weights()}, open("vectorization.pkl", "wb"))

#pickle.dump({'config': vectorization_sub.get_config(), 'weights': vectorization_sub.get_weights()}, open("vectorization_sub.pkl", "wb"))


from_disk = pickle.load(open("vectorization.pkl", "rb"))
vectorization = TextVectorization.from_config(from_disk['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization.set_weights(from_disk['weights'])

from_disk_sub = pickle.load(open("vectorization_sub.pkl", "rb"))
vectorization_sub = TextVectorization.from_config(from_disk_sub['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
vectorization_sub.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
vectorization_sub.set_weights(from_disk_sub['weights'])

2023-05-16 17:36:53.748588: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-05-16 17:36:53.857125: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


<h4>CNN</h4>

In [6]:
embedding_layer_sub = tf.keras.layers.Embedding(1000, 4)
embedding_layer = tf.keras.layers.Embedding(5000, 4)
flatten_layer = tf.keras.layers.Flatten()



def sub_processing(text, label):
    return  vectorization_sub(text), label

def processing(text, label):
    return  vectorization(text), label

train_auto_sub = train_sub.map(sub_processing)
train_auto = train.map(processing)

train_auto = train_auto.batch(32)
train_auto_sub = train_auto_sub.batch(32)


input_layer = tf.keras.layers.Input(shape=(256,))
mod = embedding_layer(input_layer)
mod =  layers.Conv1D(128, 16, activation='relu', padding='same')(mod)
mod = layers.MaxPooling1D(16, padding='same')(mod)
mod =  layers.Conv1D(128, 8, activation='relu', padding='same')(mod)
mod = layers.MaxPooling1D(8, padding='same')(mod)
mod = flatten_layer(mod)
mod =  tf.keras.layers.Dense(128, activation='relu')(mod)
mod =  tf.keras.layers.Dense(1, activation='sigmoid')(mod)

cnn = tf.keras.models.Model(input_layer, mod)

input_layer_sub = tf.keras.layers.Input(shape=(128,))
mod_sub = embedding_layer_sub(input_layer_sub)
mod_sub =  layers.Conv1D(128, 16, activation='relu', padding='same')(mod_sub)
mod_sub = layers.MaxPooling1D(16, padding='same')(mod_sub)
mod_sub =  layers.Conv1D(128, 8, activation='relu', padding='same')(mod_sub)
mod_sub = layers.MaxPooling1D(8, padding='same')(mod_sub)
mod_sub = flatten_layer(mod_sub)
mod_sub =  tf.keras.layers.Dense(128, activation='relu')(mod_sub)
mod_sub =  tf.keras.layers.Dense(1, activation='sigmoid')(mod_sub)

cnn_sub = tf.keras.models.Model(input_layer_sub, mod_sub)

# Compile and train the model
cnn.compile(optimizer='adam',  loss=losses.BinaryCrossentropy())
cnn_sub.compile(optimizer='adam',  loss=losses.BinaryCrossentropy())

import time
start_time = time.time()
cnn.fit(train_auto, epochs=10)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")
start_time = time.time()
cnn_sub.fit(train_auto_sub, epochs=10)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 1.3589859008789062e-05 seconds
Epoch 1/10


2023-05-16 17:36:54.208959: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype resource
	 [[{{node Placeholder/_9}}]]
2023-05-16 17:36:55.901276: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-05-16 17:36:56.622161: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-16 17:36:56.796666: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-16 17:36:56.800535: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0xd4b4110 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-16 17:36:56.800564: I tensorflow/

6613/6613 [==============================] - 56s 8ms/step - loss: 0.0275
Epoch 2/10
6613/6613 [==============================] - 46s 7ms/step - loss: 0.0041
Epoch 3/10
6613/6613 [==============================] - 46s 7ms/step - loss: 0.0024
Epoch 4/10
6613/6613 [==============================] - 45s 7ms/step - loss: 0.0018
Epoch 5/10
6613/6613 [==============================] - 45s 7ms/step - loss: 0.0014
Epoch 6/10
6613/6613 [==============================] - 45s 7ms/step - loss: 0.0013
Epoch 7/10
6613/6613 [==============================] - 45s 7ms/step - loss: 9.4798e-04
Epoch 8/10
6613/6613 [==============================] - 45s 7ms/step - loss: 0.0010
Epoch 9/10
6613/6613 [==============================] - 45s 7ms/step - loss: 0.0010
Epoch 10/10
6613/6613 [==============================] - 44s 7ms/step - loss: 8.4669e-04
Elapsed time: 461.02721667289734 seconds


In [8]:

#cnn.save('cnn/my_model')
#cnn_sub.save('cnn_sub/my_model')
cnn_sub = tf.keras.models.load_model('cnn_sub/my_model')
cnn = tf.keras.models.load_model('cnn/my_model')

In [9]:

test_auto_sub = test_sub.map(sub_processing)
test_auto_sub = test_auto_sub.batch(32)
test_auto = test.map(processing)
test_auto = test_auto.batch(32)
reconsturctions_sub =  cnn_sub.predict(test_auto_sub)
reconsturctions =  cnn.predict(test_auto)


2023-05-16 17:46:39.719705: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype resource
	 [[{{node Placeholder/_9}}]]


3685/3685 [==============================] - 24s 6ms/step


2023-05-16 17:47:04.783588: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype resource
	 [[{{node Placeholder/_9}}]]


3685/3685 [==============================] - 25s 7ms/step


In [10]:
pred = list(map(lambda x: 0 if x[0]<0.5 else 1, list(reconsturctions)))
pred_sub = list(map(lambda x: 0 if x[0]<0.5 else 1, list(reconsturctions_sub)))

In [11]:
actual = test_auto.map(lambda x,y:y)
actual = np.concatenate(list(actual.as_numpy_iterator()))

2023-05-16 17:47:30.518694: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype string
	 [[{{node Placeholder/_11}}]]


In [12]:
print("Metrics")
print("f1: ", f1_score(actual,pred))
print("acc: ", accuracy_score(actual,pred))
print("prec: ", precision_score(actual,pred))

Metrics
f1:  0.8311211636915242
acc:  0.7884799701439368
prec:  0.9999348205119849


In [13]:
print("Metrics sub")
print("f1: ", f1_score(actual,pred_sub))
print("acc: ", accuracy_score(actual,pred_sub))
print("prec: ", precision_score(actual,pred_sub))

Metrics sub
f1:  0.9502510823562271
acc:  0.930508316440343
prec:  0.9982012552941777
